In [1]:
import numpy as np
import pandas as pd
import pickle
from gensim.models import word2vec
import MeCab
import sys
from collections import Counter
import re
import urllib.request, urllib.error

In [2]:
tagger = MeCab.Tagger("-Owakati")
tagger.parse("") # パーサーにデータを渡す前にこれを挟むことで、UnicodeDecodeErrorを避けることが出来る。
                 #具体的な理由は分かっていないが、おそらく一度tagger.parse('')を挟むことで、プログラム内で使用されている標準の文字エンコードで初期化されるのではないかと思う。

'\n'

In [3]:
lunch_file = ["pretest_sushitsuu_lunch.csv",
              "pretest_sushiichi_lunch.csv",
              "pretest_sawada_lunch.csv",
              "pretest_mitani_lunch.csv",
              "pretest_matsukan_lunch.csv",
              "pretest_kiyoda_lunch.csv",
              "pretest_imamura_lunch.csv",
              "pretest_hatsune_lunch.csv"
              ]

In [4]:
dinner_file = ["pretest_sushitsuu_dinner.csv",
              "pretest_sushiichi_dinner.csv",
              "pretest_sawada_dinner.csv",
              "pretest_mitani_dinner.csv",
              "pretest_matsukan_dinner.csv",
              "pretest_kiyoda_dinner.csv",
              "pretest_imamura_dinner.csv",
              "pretest_hatsune_dinner.csv",
              "pretest_aozora_dinner.csv"
              ]

In [5]:
store_name = []
for i in dinner_file:
    df = pd.read_csv(i)
    store_name.append(df.iloc[0]["store_name"])

In [6]:
store_name = ["sushitsuu",
             "sushiichi",
             "sawada",
             "mitani",
             "matsukan",
             "kiyoda",
             "imamura",
             "hatsune",
             "aozora",
             ]

In [7]:
url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
slothlib_file = urllib.request.urlopen(url=url)
stop_words = [line.decode("utf-8").strip() for line in slothlib_file]
stop_words = [sw for sw in stop_words if not sw==u'']
my_stop_words = ["℃/", "℃", "kg", "訪問", "寿司", "すし", "なく", "美味しい", "美味", "～」", "○－", "○○/", ")○/"]
stop_words += my_stop_words 

In [8]:
def normalize_number(text):
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text

In [9]:
def tokenize_ja(text, lower):
    text = replaced_text = re.sub(r"\d+", "0", text)
    node = tagger.parseToNode(str(text))
    while node:
        surface = node.surface.lower()
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"] and surface not in stop_words:
            #分かち書きで取得する品詞を指定
            yield surface
            #lowerで小文字に変換 正規化
        node = node.next

In [10]:
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [11]:
def kyouki_lunch(path):

    df = pd.read_csv(path)
    wakati_sushi_text = []
    word_list = []
    many_words = []
    similar_words = []
    for i in df["lunch_review"]:
        txt = tokenize(i, 2, 10000, True)
        wakati_sushi_text.append(txt)
        for t in txt:
            word_list.append(t)
    #word2vec_sushi_model = word2vec.Word2Vec(word_list,sg=1,size=100, window=5,min_count=5,iter=100,workers=3)
    print("昼間" in word_list)
    word2vec_sushi_model = word2vec.Word2Vec(wakati_sushi_text,sg=1,size=100, window=5,min_count=4,iter=100,workers=3)
    counter = Counter(word_list)
    for word, count in counter.most_common(10):
        #word2vec_sushi_model = word2vec.Word2Vec(wakati_sushi_text,sg=1,size=100, window=5,min_count=5,iter=100,workers=3)
        many_words.append(word)
        print(word)
        print(count)
        similar = word2vec_sushi_model.most_similar(word)
        similar_words.append(similar)
    return many_words, similar_words

In [12]:
def kyouki_dinner(path):
    df = pd.read_csv(path)
    wakati_sushi_text = []
    word_list = []
    many_words = []
    similar_words = []
    for i in df["dinner_review"]:
        txt = tokenize(i, 2, 10000, True)
        wakati_sushi_text.append(txt)
        word_list.append(i)
        for t in txt:
            word_list.append(t)
    counter = Counter(word_list)
    for word, count in counter.most_common(10):
        many_words.append(word)
    word2vec_sushi_model = word2vec.Word2Vec(wakati_sushi_text,sg=1,size=100, window=5,min_count=3,iter=100,workers=3)
    for many in many_words:
        #word2vec_sushi_model = word2vec.Word2Vec(wakati_sushi_text,sg=1,size=100, window=5,min_count=5,iter=100,workers=3)
        similar = word2vec_sushi_model.most_similar(many)
        similar_words.append(similar)
    return many_words, similar_words

In [13]:
many_words_lunch = []
similar_words_lunch = []
for path in lunch_file:
    many_word, similar_word = kyouki_lunch(path)
    many_words_lunch.append(many_word)
    similar_words_lunch.append(similar_word)
    print("=====================")

True
握り
196
熟成
175
ランチ
139
ネタ
131
シャリ
125
大将
111
予約
98
海老
81
トロ
76
カウンター
75
True


/Users/yuki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


銀座
52
握り
52
ランチ
47
コース
46
カウンター
30
シャリ
30
穴子
26
料理
25
良い
25
いい
24
True
トロ
168
握り
143
シャリ
116
さわ
104
ネタ
89
穴子
70
澤田
68
ない
66
雲丹
65
軍艦
63
True
三谷
182
予約
151
料理
113
握り
103
ワイン
69
雲丹
56
日本
53
トロ
50
穴子
47
巻き
47
True
握り
12
ネタ
10
まつ
7
予約
7
トロ
7
麻布
7
雰囲気
6
穴子
5
店内
5
昼間
4
False
マグロ
39
トロ
32
握り
25
きよ
23
赤身
21
旨味
17
巻き
14
最高
13
ない
12
まぐろ
11
True
握り
90
シャリ
59
ネタ
53
穴子
50
良い
49
トロ
49
車海老
40
ない
39
予約
35
いい
35
False
シャリ
91
トロ
84
握り
81
ネタ
79
漬け
65
大将
55
旨み
54
初音
47
赤身
42
香り
42


In [14]:
similar_words_lunch

[[[('展開', 0.4852670431137085),
   ('お昼', 0.4426043629646301),
   ('休日', 0.41934096813201904),
   ('定番', 0.40276628732681274),
   ('売り切れ', 0.37955278158187866),
   ('満席', 0.372326135635376),
   ('邪魔', 0.37155264616012573),
   ('美術館', 0.3682275712490082),
   ('ボリューム', 0.3676299750804901),
   ('シャリ', 0.35677722096443176)],
  [('日間', 0.439258873462677),
   ('熱心', 0.4146321713924408),
   ('一言', 0.40525734424591064),
   ('別物', 0.39941301941871643),
   ('成分', 0.3919495940208435),
   ('ムース', 0.37236449122428894),
   ('ヅケ', 0.3700936436653137),
   ('口中', 0.3626708984375),
   ('変態', 0.35227885842323303),
   ('旨味', 0.350339412689209)],
  [('休日', 0.4635220766067505),
   ('予約', 0.4202212691307068),
   ('投稿', 0.4155302047729492),
   ('定食', 0.41287386417388916),
   ('久々', 0.4124143719673157),
   ('ログ', 0.40765583515167236),
   ('開店', 0.3938494920730591),
   ('美術館', 0.3929765224456787),
   ('焼き魚', 0.3773401975631714),
   ('シャンパン', 0.3770952820777893)],
  [('シャリ', 0.46661391854286194),
   ('大き', 0.4183

In [15]:
many_words_lunch

[['握り', '熟成', 'ランチ', 'ネタ', 'シャリ', '大将', '予約', '海老', 'トロ', 'カウンター'],
 ['銀座', '握り', 'ランチ', 'コース', 'カウンター', 'シャリ', '穴子', '料理', '良い', 'いい'],
 ['トロ', '握り', 'シャリ', 'さわ', 'ネタ', '穴子', '澤田', 'ない', '雲丹', '軍艦'],
 ['三谷', '予約', '料理', '握り', 'ワイン', '雲丹', '日本', 'トロ', '穴子', '巻き'],
 ['握り', 'ネタ', 'まつ', '予約', 'トロ', '麻布', '雰囲気', '穴子', '店内', '昼間'],
 ['マグロ', 'トロ', '握り', 'きよ', '赤身', '旨味', '巻き', '最高', 'ない', 'まぐろ'],
 ['握り', 'シャリ', 'ネタ', '穴子', '良い', 'トロ', '車海老', 'ない', '予約', 'いい'],
 ['シャリ', 'トロ', '握り', 'ネタ', '漬け', '大将', '旨み', '初音', '赤身', '香り']]

In [16]:
many_words_dinner = []
similar_words_dinner = []
for path in dinner_file:
    many_word, similar_word = kyouki_dinner(path)
    many_words_dinner.append(many_word)
    similar_words_dinner.append(similar_word)
    print("=====================")

/Users/yuki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [17]:
many_words_dinner

[['握り', '大将', '熟成', 'ネタ', 'トロ', '包丁', 'シャリ', '料理', '予約', 'カウンター'],
 ['銀座', '握り', 'カウンター', 'ワイン', '予約', 'コース', 'いい', '良い', 'ネタ', '雰囲気'],
 ['トロ', '握り', 'さわ', 'シャリ', '銀座', 'ネタ', '穴子', '大将', 'ない', '雲丹'],
 ['三谷', '予約', '料理', '握り', 'ワイン', '日本', 'マリアージュ', '雲丹', 'ない', '大将'],
 ['カウンター', '握り', '予約', 'いい', '値段', '麻布', '良い', 'ない', 'ネタ', '好き'],
 ['マグロ', '握り', 'きよ', 'まぐろ', 'トロ', '最高', '銀座', '旨い', '穴子', '大将'],
 ['握り', 'シャリ', 'トロ', '穴子', '良い', 'ネタ', '白金', '車海老', '大将', 'カウンター'],
 ['トロ', '握り', 'シャリ', '大将', 'ネタ', '初音', '漬け', '赤身', '予約', 'マグロ'],
 ['握り', 'シャリ', '穴子', 'トロ', '銀座', '青空', '大将', 'ネタ', 'すき', 'やばし']]

In [18]:
similar_words_dinner

[[[('毛蟹', 0.3899323344230652),
   ('タコマンマ', 0.3692707419395447),
   ('メニュー', 0.3680764138698578),
   ('紫蘇', 0.36470162868499756),
   ('あん', 0.35857969522476196),
   ('アラ', 0.35778623819351196),
   ('墨烏賊', 0.3574782609939575),
   ('主役', 0.3559643030166626),
   ('計算', 0.3498983085155487),
   ('半生', 0.34957176446914673)],
  [('努力', 0.4269537925720215),
   ('予約', 0.40421849489212036),
   ('台湾', 0.3971474766731262),
   ('欲しい', 0.37969499826431274),
   ('商売', 0.3792075514793396),
   ('っ子', 0.37747254967689514),
   ('江戸', 0.372334361076355),
   ('ジャンル', 0.36986052989959717),
   ('熱心', 0.36603182554244995),
   ('笑顔', 0.36273741722106934)],
  [('キロ', 0.46646562218666077),
   ('圧巻', 0.4158507287502289),
   ('凝縮', 0.40498095750808716),
   ('極限', 0.4041348099708557),
   ('新鮮', 0.389718234539032),
   ('ネットリ', 0.38683265447616577),
   ('カット', 0.3859863579273224),
   ('曰く', 0.38393789529800415),
   ('隠し', 0.3804023861885071),
   ('たて', 0.3676767945289612)],
  [('小さめ', 0.4370644986629486),
   ('解説', 0

In [19]:

#df = pd.read_csv('pretest_matsukan_lunch.csv')
##df_sushi = df.groupby(['store_name','score'])['review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)
##コーパス作成
#wakati_sushi_text = []
#word_list = []
#for i in df["lunch_review"]:
    #txt = tokenize(i, 2, 10000, True)
    #wakati_sushi_text.append(txt)
## モデル作成
#word2vec_sushi_model = word2vec.Word2Vec(wakati_sushi_text,sg=1,size=100, window=5,min_count=5,iter=100,workers=3)
##sg（0: CBOW, 1: skip-gram）,size（ベクトルの次元数）,window（学習に使う前後の単語数）,min_count（n回未満登場する単語を破棄）,iter（トレーニング反復回数）
##workers:複数のスレッドで処理

In [20]:
def makedf_lunch(i):
    df = pd.DataFrame({many_words_lunch[i][0]:similar_words_lunch[i][0]})
    for t in range(1, len(store_name)-1):
        df2 = pd.DataFrame({many_words_lunch[i][t]:similar_words_lunch[i][t]})
        df = pd.concat([df, df2], axis=1)
    return df

In [21]:
lunch = []
for i in range(len(store_name) - 1): 
    lunch.append(makedf_lunch(i))

In [22]:
#for i in range(len(lunch)):
    #lunch[i].to_csv("pre_sentiment_"+store_name[i]+"_lunch.csv")

In [23]:
similar_words_lunch_l = []
for i in range(len(similar_words_lunch)):
    ll = []
    for s in range(len(similar_words_lunch[i])):
        l = []
        for t in range(len(similar_words_lunch[i][s])):
            l.append(similar_words_lunch[i][s][t][0])
        ll.append(l)
    similar_words_lunch_l.append(ll)

In [24]:
def makedf_lunch(i):
    df = pd.DataFrame({many_words_lunch[i][0]:similar_words_lunch_l[i][0]})
    for t in range(1, len(similar_words_lunch_l[i])):
        df2 = pd.DataFrame({many_words_lunch[i][t]:similar_words_lunch_l[i][t]})
        df = pd.concat([df, df2], axis=1)
    return df

In [25]:
lunch = []
for i in range(len(similar_words_lunch_l)): 
    lunch.append(makedf_lunch(i))

In [26]:
for i in range(len(lunch)):
    lunch[i].to_csv("pre_frequency_"+store_name[i]+"_lunch.csv", index=False)

In [27]:
similar_words_dinner_l = []
for i in range(len(similar_words_dinner)):
    ll = []
    for s in range(len(similar_words_dinner[i])):
        l = []
        for t in range(len(similar_words_dinner[i][s])):
            l.append(similar_words_dinner[i][s][t][0])
        ll.append(l)
    similar_words_dinner_l.append(ll)

In [28]:
def makedf_dinner(i):
    df = pd.DataFrame({many_words_dinner[i][0]:similar_words_dinner_l[i][0]})
    for t in range(1, len(similar_words_dinner_l[i])):
        df2 = pd.DataFrame({many_words_dinner[i][t]:similar_words_dinner_l[i][t]})
        df = pd.concat([df, df2], axis=1)
    return df

In [29]:
dinner = []
for i in range(len(similar_words_dinner_l)): 
    dinner.append(makedf_dinner(i))

In [30]:
for i in range(len(dinner)):
    dinner[i].to_csv("pre_frequency_"+store_name[i]+"_dinner.csv", index=False)

In [32]:
df = pd.read_csv("pre_frequency_sushitsuu_lunch.csv")

In [33]:
df

,握り,熟成,ランチ,ネタ,シャリ,大将,予約,海老,トロ,カウンター
0,展開,日間,休日,シャリ,ネタ,食事,到着,雲丹,スタート,個室
1,お昼,熱心,予約,大き,小さめ,是非,会社,ボタン,ブリ,広い
2,休日,一言,投稿,小さめ,固め,驚き,電話,富山,富山,店内
3,定番,別物,定食,飾り,大き,っ子,ログ,穴子,大間,入口
4,売り切れ,成分,久々,満腹,主張,若い,当日,山盛り,残念,清潔
5,満席,ムース,ログ,鮮度,ズワイ,英語,入口,強く,舌触り,地下
6,邪魔,ヅケ,開店,基本,玉子,台湾,時半,香ばし,ネギ,満席
7,美術館,口中,美術館,一体,しま,うまく,売り切れ,ズワイ,包丁,時半
8,ボリューム,変態,焼き魚,工夫,ブレンド,予約,お客,西京,ムース,後ろ
9,シャリ,旨味,シャンパン,別物,富山,邪魔,平日,ジューシー,カワハギ,案内


In [ ]:
df